# Подключение библиотек и обучающих данных

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# import os
# os.chdir("drive/My Drive/data_autopilot")

In [3]:
# !ls

In [4]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
#from google.colab.patches import cv2_imshow

In [5]:
import os
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]


In [6]:
from PIL import Image

In [7]:
# tf.test.is_built_with_cuda()
# tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

In [8]:
from keras.models import Sequential
from keras.models import load_model

from keras.layers import Flatten, Dense
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import concatenate, Concatenate
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.optimizers import Adam
from keras.preprocessing import image
from keras.callbacks import CSVLogger, ModelCheckpoint

In [9]:
# import tensorflow as tf
# from tensorflow.python.keras import backend as K

# config = tf.compat.v1.ConfigProto
# config.gpu_options.per_process_gpu_memory_fraction = 0.75
# sess = tf.compat.v1.Session()
# K.set_session(sess)

In [10]:
IMAGE_WIDTH=375
IMAGE_HEIGHT=620
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)

In [11]:
main_dir_path = 'C:\Projects\data_autopilot'

csv_file = 'v1'
csv_file_dir = os.path.join(main_dir_path, 'csv', 'final')
img_front_dir_path = os.path.join(main_dir_path, 'img', 'first_data')

train_file = os.path.join(csv_file_dir, csv_file + '_train.csv')
valid_file = os.path.join(csv_file_dir, csv_file + '_valid.csv')

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\dan4i\AppData\Local\Temp\ipykernel_16092\143837886.py:1: SyntaxWarning: invalid escape sequence '\P'
  main_dir_path = 'C:\Projects\data_autopilot'


In [12]:
# divide by steer output to [-100, 100]
OUTPUT_NORMALIZATION = 100

In [13]:
df_train = pd.read_csv(train_file)

print(f'Records - {df_train.shape[0]}')

df_train.head(3)

Records - 21533


,img,steer,throttle,brake,angle_path
0,autopilot_data_2024_05_20_12_58_52_58.jpg,224,4164,0,6
1,autopilot_data_2024_05_20_12_50_20_16.jpg,-1669,9991,0,31
2,autopilot_data_2024_05_20_13_30_06_43.jpg,3,0,0,0


In [14]:
df_val = pd.read_csv(valid_file)

print(f'Records - {df_val.shape[0]}')

df_val.head(3)

Records - 3801


,img,steer,throttle,brake,angle_path
0,autopilot_data_2024_05_20_13_17_54_32.jpg,-109,5549,0,89
1,autopilot_data_2024_05_20_13_13_30_78.jpg,-48,6958,0,89
2,autopilot_data_2024_05_20_13_25_12_28.jpg,1,3792,0,90


In [15]:
df_train['steer'] = df_train['steer'] // OUTPUT_NORMALIZATION

In [16]:
df_train.to_csv(train_file, index=False)

## Обучение

In [17]:
def img_to_arr(p):
  with image.load_img(p) as img:
      img = image.img_to_array(img)

  return img

In [18]:
# def normalize(img):

#   img[:,:,0] -= 52.5147
#   img[:,:,0] /= 34.2709

#   img[:,:,1] -= 52.4108
#   img[:,:,1] /= 31.9759

#   img[:,:,2] -= 47.1400
#   img[:,:,2] /= 30.3161

#   return img

In [19]:
# def generator(df, batch_size, img_shape, should_shuffle):
#     if should_shuffle:
#         df = shuffle(df)

#     img_list = df['img']
#     wheel_axis = df['steer']

#     batch_img = np.zeros((batch_size,) + img_shape)
#     batch_label = np.zeros((batch_size, 1))

#     index = 0
#     while True:
#         for i in range(batch_size):
#             img_name = img_list[index]
#             arr = img_to_arr(os.path.join(img_front_dir_path, img_name))

#             batch_img[i] = normalize(arr)
#             batch_label[i] = wheel_axis[index] / OUTPUT_NORMALIZATION

#             index += 1
#             if index == len(img_list):
#                 index = 0

#         yield batch_img, batch_label

In [20]:
# train_batch = generator(df_train, batch_size, input_shape, True)
# val_batch = generator(df_val, batch_size, input_shape, False)

In [21]:
input_shape = img_to_arr(os.path.join(img_front_dir_path, df_train['img'][0])).shape
batch_size = 150
train_steps = int((df_train.shape[0] / batch_size) + 1)
val_steps = int((df_val.shape[0] / batch_size) + 1)

print("input_shape: %s, batch_size: %d, train_steps: %d, val_steps: %d" %
      (input_shape, batch_size, train_steps, val_steps))

input_shape: (375, 620, 3), batch_size: 150, train_steps: 144, val_steps: 26


https://keras.io/guides/functional_api/

In [22]:
def get_model(input_shape):
    model = Sequential([
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu', input_shape=input_shape),
        BatchNormalization(axis=1),

        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),

        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(axis=1),

        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),

        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(axis=1),

        Flatten(),

        Dense(70, activation='relu'),
        BatchNormalization(),

        Dense(30, activation='relu'),
        BatchNormalization(),

        Dense(10, activation='relu'),
        BatchNormalization(),

        Dense(1)
    ])

    return model

In [23]:
model = get_model(input_shape)
#sgd = SGD(learning_rate=1e-3, momentum=0.9, nesterov=True)
model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse')
model.summary()

c:\Projects\Cursovik\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 186, 308, 24)   │         1,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 186, 308, 24)   │           744 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 91, 152, 36)    │        21,636 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 91, 152, 36)    │           364 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 44, 74, 48)     │        43,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 44, 74, 48)     │           176 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 42, 72, 64)     │        27,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 42, 72, 64)     │           168 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 40, 70, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 40, 70, 64)     │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 179200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 70)             │    12,544,070 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 70)             │           280 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         2,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 30)             │           120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           310 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,679,921 (48.37 MB)

 Trainable params: 12,678,895 (48.37 MB)

 Non-trainable params: 1,026 (4.01 KB)

In [24]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_dataframe(
    df_train, 
    img_front_dir_path,
    x_col='img',
    y_col='steer',
    target_size=IMAGE_SIZE,
    class_mode='raw',
    batch_size=batch_size
)

Found 21533 validated image filenames.


In [25]:
validation_datagen = ImageDataGenerator(
    rescale=1./255,
)

validation_generator = validation_datagen.flow_from_dataframe(
    df_val, 
    img_front_dir_path,
    x_col='img',
    y_col='steer',
    target_size=IMAGE_SIZE,
    class_mode='raw',
    batch_size=batch_size
)

Found 3801 validated image filenames.


In [26]:
history = model.fit(train_generator,
                    steps_per_epoch = train_steps,
                    epochs=20,
                    verbose=1,
                    validation_data=validation_generator,
                    validation_steps=val_steps)

Epoch 1/20


c:\Projects\Cursovik\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


144/144 ━━━━━━━━━━━━━━━━━━━━ 985s 7s/step - loss: 46.4914 - val_loss: 505172.0000
Epoch 2/20


C:\Users\dan4i\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


AttributeError: 'NoneType' object has no attribute 'items'

## Другой вариант

In [ ]:
# def get_model(input_shape):

#   img_input = Input(
#     shape=input_shape, name="img"
#   )
#   angle_input = Input(
#       shape=(1,), name="angle"
#   )

#   conv_layer = Conv2D(24, kernel_size=(5, 5), strides=(2, 2), activation='relu')(img_input)
#   conv_layer = BatchNormalization(axis=1)(conv_layer)

#   conv_layer = Conv2D(36, kernel_size=(5, 5), strides=(2, 2), activation='relu')(conv_layer)
#   conv_layer = BatchNormalization(axis=1)(conv_layer)

#   conv_layer = Conv2D(48, kernel_size=(5, 5), strides=(2, 2), activation='relu')(conv_layer)
#   conv_layer = BatchNormalization(axis=1)(conv_layer)

#   conv_layer = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu')(conv_layer)
#   conv_layer = BatchNormalization(axis=1)(conv_layer)

#   conv_layer = Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu')(conv_layer)
#   conv_layer = BatchNormalization(axis=1)(conv_layer)

#   merged_layer = Flatten()(conv_layer)
#   merged_layer = Dense(70, activation='relu')(merged_layer)

#   merged_layer = concatenate([merged_layer, angle_input])
#   merged_layer = BatchNormalization()(merged_layer)

#   merged_layer = Dense(30, activation='relu')(merged_layer)
#   merged_layer = BatchNormalization()(merged_layer)

#   merged_layer = Dense(10, activation='relu')(merged_layer)
#   merged_layer = BatchNormalization()(merged_layer)

#   output = Dense(3, activation='linear', name='output')(merged_layer)

#   model = Model(
#     inputs=[img_input, angle_input],
#     outputs={"output": output},
#   )

#   return model

In [ ]:
# keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

In [ ]:
# def generator_data(df, img_shape, should_shuffle, kol_data):
#     if should_shuffle:
#         df = shuffle(df)

#     img_list = df['img']
#     angle_list = df['angle_path']
#     steer_list = df['steer']
#     throttle_list = df['throttle']
#     brake_list = df['brake']

#     # create empty batch
#     img_data = np.zeros((kol_data,) + img_shape)
#     output_data = np.zeros((kol_data, 3))
#     angle_data = np.zeros((kol_data, 1))

#     # print(img_list)
#     # print(angle_list)

#     index = 0
#     for i in range(kol_data):
#         img_name = img_list[i]
#         arr = img_to_arr(os.path.join(img_front_dir_path, img_name))

#         img_data[i] = normalize(arr)
#         output_data[i] = [steer_list[i], throttle_list[i], brake_list[i]]
#         angle_data[i] = angle_list[i]

#         print(i)

#     return img_data, output_data, angle_data

In [ ]:
# img_data, output_data, angle_data = generator_data(df_train, input_shape, True, 1000)

In [ ]:
# print(img_data.shape)
# print(output_data.shape)
# print(angle_data.shape)

In [ ]:
# model.fit(
#     {"img": img_data, "angle": angle_data},
#     {"output": output_data},
#     epochs=2,
#     batch_size=150,
# )